In [1]:
import pandas as pd
import preprocess
import spacy_stanza
from spacytextblob.spacytextblob import SpacyTextBlob
import re

nlp = spacy_stanza.load_pipeline("en", use_gpu=False)
import swifter

%config Completer.use_jedi = False

2022-02-21 08:53:16 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2022-02-21 08:53:16 INFO: Use device: cpu
2022-02-21 08:53:16 INFO: Loading: tokenize
2022-02-21 08:53:16 INFO: Loading: pos
2022-02-21 08:53:16 INFO: Loading: lemma
2022-02-21 08:53:16 INFO: Loading: depparse
2022-02-21 08:53:16 INFO: Loading: sentiment
2022-02-21 08:53:16 INFO: Loading: constituency
2022-02-21 08:53:16 INFO: Loading: ner
2022-02-21 08:53:17 INFO: Done loading processors!
/home/kr/anaconda3/envs/nlp/lib/python3.8/site-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int

In [16]:
df = pd.read_json('yulu.gplay.json')[['reviewId', 'at', 'content']]
df = df.head(8000)

In [22]:
df['phrases'] = df.content.swifter.apply(lambda x: preprocess.preprocess(x, True, nlp).phrase_extract())

Pandas Apply:   0%|          | 0/8000 [00:00<?, ?it/s]

In [60]:
tupled = list(zip(df.reviewId.to_list(), df['at'].to_list(), df.content.to_list(), df.phrases.to_list()))

In [69]:
expanded_table = []
for tuples in tupled:
    for phrase in tuples[3]:
        if len(phrase.split(' ')) == 2:
            expanded_table.append(
                (                
                    tuples[0], 
                    tuples[1], 
                    tuples[2], 
                    phrase.split(' ')[0], 
                    phrase.split(' ')[1]
                )
        )

In [115]:
df_out = pd.DataFrame(expanded_table, columns=['ID', 'timestamp', 'comment', 'assessment', 'target'])

In [116]:
df_out['target'] = df_out.target.swifter.apply(lambda x: x.lower())
df_out['assessment'] = df_out.assessment.swifter.apply(lambda x: x.lower())

Pandas Apply:   0%|          | 0/35121 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/35121 [00:00<?, ?it/s]

In [117]:
df_out.to_csv('ref_table.csv', index = False)

In [89]:
set_app_ass = set(df_out[df_out.target == 'app'].assessment.to_list())

In [109]:
dict_app_ass = {}
for key in set_app_ass:
    dict_app_ass[key] = 0
for ass in df_out[df_out.target == 'app'].assessment.to_list():
    dict_app_ass[ass] = dict_app_ass[ass] + 1
dict_app_ass = dict(sorted(dict_app_ass.items(), key=lambda x:x[1], reverse = True))

In [110]:
dict_app_ass

{'nice': 79,
 'worst': 74,
 'good': 64,
 'use': 56,
 'is': 37,
 'best': 30,
 'open': 26,
 'download': 25,
 'great': 24,
 'it': 24,
 'your': 22,
 ',': 19,
 'friendly': 19,
 'showing': 18,
 'using': 18,
 'ever': 17,
 'bad': 17,
 'easy': 16,
 'opening': 16,
 'working': 14,
 'awesome': 13,
 'experience': 13,
 'service': 13,
 'love': 12,
 'my': 12,
 '..': 12,
 'and': 12,
 'slow': 11,
 'work': 11,
 'install': 11,
 'shows': 11,
 'helpful': 10,
 'update': 10,
 "'s": 10,
 'this': 10,
 'issue': 10,
 'show': 10,
 'excellent': 10,
 'amazing': 9,
 'like': 9,
 '...': 9,
 'useful': 9,
 '....': 9,
 'useless': 9,
 'uninstall': 9,
 'ride': 8,
 'need': 8,
 '!': 7,
 'waste': 7,
 'user': 7,
 'so': 7,
 'thanks': 7,
 'used': 7,
 '': 6,
 'yulu': 6,
 'app': 6,
 'keeps': 6,
 'end': 6,
 'one': 5,
 'its': 5,
 'seen': 5,
 'not': 5,
 'superb': 5,
 'fake': 5,
 'asking': 5,
 'pathetic': 5,
 'begar': 5,
 'got': 4,
 'other': 4,
 'liked': 4,
 'poor': 4,
 'ui': 4,
 'simple': 4,
 'beautiful': 4,
 'has': 4,
 'fraud': 4,
 '

{'nice': 79,
 'worst': 74,
 'good': 64,
 'use': 56,
 'is': 37,
 'best': 30,
 'open': 26,
 'download': 25,
 'great': 24,
 'it': 24,
 'your': 22,
 ',': 19,
 'friendly': 19,
 'showing': 18,
 'using': 18,
 'ever': 17,
 'bad': 17,
 'easy': 16,
 'opening': 16,
 'working': 14,
 'awesome': 13,
 'experience': 13,
 'service': 13,
 'love': 12,
 'my': 12,
 '..': 12,
 'and': 12,
 'slow': 11,
 'work': 11,
 'install': 11,
 'shows': 11,
 'helpful': 10,
 'update': 10,
 "'s": 10,
 'this': 10,
 'issue': 10,
 'show': 10,
 'excellent': 10,
 'amazing': 9,
 'like': 9,
 '...': 9,
 'useful': 9,
 '....': 9,
 'useless': 9,
 'uninstall': 9,
 'ride': 8,
 'need': 8,
 '!': 7,
 'waste': 7,
 'user': 7,
 'so': 7,
 'thanks': 7,
 'used': 7,
 '': 6,
 'yulu': 6,
 'app': 6,
 'keeps': 6,
 'end': 6,
 'one': 5,
 'its': 5,
 'seen': 5,
 'not': 5,
 'superb': 5,
 'fake': 5,
 'asking': 5,
 'pathetic': 5,
 'begar': 5,
 'got': 4,
 'other': 4,
 'liked': 4,
 'poor': 4,
 'ui': 4,
 'simple': 4,
 'beautiful': 4,
 'has': 4,
 'fraud': 4,
 '

In [102]:
dict_app_ass

{'': 0,
 'wiped': 0,
 'showing': 0,
 'load': 0,
 'confidence': 0,
 'ask': 0,
 'woset': 0,
 'fudu': 0,
 'loaded': 0,
 'user-friendly': 0,
 'placement': 0,
 'should': 0,
 'rs': 0,
 'request': 0,
 'faced': 0,
 'idiotic': 0,
 'very': 0,
 'be': 0,
 'logout': 0,
 '&': 0,
 'add': 0,
 'slow': 0,
 'operate': 0,
 'pura': 0,
 'stopped': 0,
 'launched': 0,
 'hole': 0,
 'those': 0,
 'support': 0,
 'literate': 0,
 'stupid': 0,
 'helpful': 0,
 'speed': 0,
 'enter': 0,
 '!': 0,
 'called': 0,
 'easy': 0,
 'west': 0,
 'yulu': 0,
 'only': 0,
 '.very': 0,
 'one': 0,
 'updated': 0,
 'behave': 0,
 'performance': 0,
 'fantastic': 0,
 'let': 0,
 'look': 0,
 'hope': 0,
 'marred': 0,
 '2': 0,
 'wrote': 0,
 'thuuu': 0,
 'there': 0,
 'care': 0,
 'doing': 0,
 'ue': 0,
 'improvement': 0,
 'disgusting': 0,
 'got': 0,
 'recurring': 0,
 'other': 0,
 'stucked': 0,
 'thing': 0,
 'pause': 0,
 'active': 0,
 'nice': 0,
 'for': 0,
 'least': 0,
 'sickening': 0,
 'handy': 0,
 'locate': 0,
 'work': 0,
 'seem': 0,
 'problematic

In [141]:
print(df_out.head(35)[df_out.columns[1:]].to_markdown())

|    | timestamp           | comment                                                                                                                                                                                  | assessment   | target     |
|---:|:--------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-------------|:-----------|
|  0 | 2021-08-27 03:24:25 | App is crashing like anything even after updating it..                                                                                                                                   | crashing     | app        |
|  1 | 2021-08-27 03:24:25 | App is crashing like anything even after updating it..                                                                                                                                   | crashing     | anything   |
|  2 | 2021-08-26 19:14: